# Yahoo 電影爬蟲練習：爬取電影放映查詢結果
1. 獲取電影的代號、放映地區、放映日期
2. 再送出查詢給伺服器

Hint: 
- 片名列表 API 位在 `<select>` 標籤內的 `data-url` 


In [10]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests
from tqdm import tqdm

### 先搜尋全部的電影代號(ID)資訊

In [13]:
import requests

url = "https://movies.yahoo.com.tw/movie_intheaters.html"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

res = requests.get(url, headers=headers)
res.encoding = 'utf-8'

print(res.text)  # 輸出 HTML，手動檢查是否有 class="release_box"


<!DOCTYPE html><html id="atomic" class="NoJs chrome desktop" lang="zh-Hant-TW"><head prefix="og: http://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark('PageStart');</script><meta charSet="utf-8"/><meta name="referrer" content="origin"/><meta name="theme-color" content="#400090"/><meta name="page" content="娛樂分類頁"/><meta name="oath:guce:consent-host" content="guce.yahoo.com"/><meta name="twitter:description" content="Yahoo奇摩娛樂新聞讓你掌握兩岸三地、歐美、日韓各地藝人最新動態，聚焦多元、專業、平衡的分析報導。"/><meta name="twitter:image" content="https://s.yimg.com/cv/apiv2/social/images/yahoo_default_logo-1200x1200.png"/><meta name="twitter:image:src" content="https://s.yimg.com/cv/apiv2/social/images/yahoo_default_logo-1200x1200.png"/><meta name="twitter:title" content="娛樂 - Yahoo奇摩新聞"/><meta name="description" content="Yahoo奇摩娛樂新聞讓你掌握兩岸三地、歐美、日韓各地藝人最新動態，聚焦多元、專業、平衡的分析報導。"/><meta name="keywords" content="娛樂, 影劇, 狗仔, 明星, 名模, 偶像劇, 歌手, 韓劇, 日劇, 金鐘獎, 金馬獎, 金曲獎, 藝人微博, 韓流瘋, 追劇, 娛樂新聞, 財經頭條新聞, 

In [ ]:
url = "https://movies.yahoo.com.tw/ajax/in_theater_movies"
movie_ids = requests.get(url).json()
movie_ids

movie_id = 10159


In [ ]:
url = "https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater"
payload = {'movie_id': str(movie_id)}

# headers 中要有 mv-authorization 才能得到回傳結果，你可以試著把 mv-authorization 拿掉試試看會回傳什麼訊息
# 其他的資訊可有可無，不過為了讓學員學好學滿，還是附上完整的 headers 資訊會長怎樣
headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
res = requests.get(url, params=payload, headers=headers).json()
print(res)
# 整理放映地區資料
publish_area = {item["title"]: item["area_id"] for item in res}
print(publish_area)

In [ ]:
# 指定放映地區
area_id = publish_area["台北市"]
# 向網站發送請求
url = "https://movies.yahoo.com.tw/movietime_result.html"
payload = {"movie_id": str(movie_id), "area_id": str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = "utf-8"

soup = BeautifulSoup(resp.text, 'html.parser')
movie_dates = soup.find_all("label", attrs={'for': re.compile("date_[\d]")})  # 利用 regex 篩選符合條件的 <label></label> 節點
movie_dates = [date.p.string + " " + date.h3.string for date in movie_dates]
print(movie_dates)

In [ ]:
# 向網站發送請求
date = "2022-06-11"
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {
    "movie_id": str(movie_id),
    "date": date,
    "area_id": str(area_id),
    "theater_id": "",
    "datetime": "",
    "movie_type_id": ""
}

res = requests.get(url, params=payload).json()
soup = BeautifulSoup(res["view"], "html.parser")
html = soup.find_all("ul", attrs={'data-theater_name': re.compile(".*")})
print(html)

for movie_theater in html:
    print("----------------------------------------------------------------------")
    
    theater_name = movie_theater.find("li", attrs={'class':'adds'})
    print("電影院: ", theater_name.a.string)

    project_type = movie_theater.find_all("li", attrs={'class':'taps'})  # 定位放映類型
        
    time_table = movie_theater.find_all("li", attrs={'class':'time _c'})  # 定位時間表

    for i in range(len(project_type)):
        print("放映類型: ", project_type[i].span.get_text())  # 印出電影類型
        
        for p4 in time_table[i].find_all("input", attrs={'name':'schedule_list'}):
            print("%s" % (p4["value"]))  # 印出放映時間